In [3]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [12]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (test2!=0):
        t2=t2+[a]
    if (test3!=0):
        t3=t3+[a]
    if mod(a,10)==0:print(a)
print();print(t2);print();print(t3)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[1, 3, 7, 9, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 149, 151, 153, 155, 157, 159, 161, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 209, 211, 213, 215, 217, 219, 221, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 277, 27

In [15]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (mod(a,2)==0)&(test2!=0):
        t2=t2+[a]
    if (mod(a,3)==0)&(test3!=0):
        t3=t3+[a]
    if mod(a,10)==0:print(a)
print();print(t2);print();print(t3)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[]

[]


In [16]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if ((test2==0)&(mod(a,2)!=0)):
        t2=t2+[a]
    if ((test3==0)&(mod(a,3)!=0)):
        t3=t3+[a]
    if mod(a,10)==0:print(a)
print();print(t2);print();print(t3)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[5, 11, 83, 147, 163, 207, 223, 275, 291, 303, 323, 399, 531, 547, 559, 579, 643, 679, 703, 783]

[1, 2, 4, 5, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 22, 23, 25, 26, 28, 29, 31, 32, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47, 49, 50, 52, 53, 55, 56, 58, 59, 61, 62, 64, 65, 67, 68, 70, 71, 73, 74, 76, 77, 79, 80, 82, 83, 85, 86, 88, 89, 91, 92, 94, 95, 97, 98, 100, 101, 103, 104, 106, 107, 109, 110, 112, 113, 115, 116, 118, 119, 121, 122, 124, 125, 127, 128, 130, 131, 133, 134, 136, 137, 139, 140, 142, 143, 145, 146, 148, 149, 151, 152, 154, 155, 157, 158, 160, 161, 163, 164, 166, 167, 169, 170, 172, 173, 

In [17]:
print(difference_table(t2))

[6, 72, 64, 16, 44, 16, 52, 16, 12, 20, 76, 132, 16, 12, 20, 64, 36, 24, 80]


In [19]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)
ls=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    ds2=digitsum(2,a)
    test=3*ds2-o2
    if (test==0)&(mod(a,2)==1):
        print((a,mod(a,4)))
        ls=ls+[a]
print();print(ls)

(5, 1)
(11, 3)
(83, 3)
(147, 3)
(163, 3)
(207, 3)
(223, 3)
(275, 3)
(291, 3)
(303, 3)
(323, 3)
(399, 3)
(531, 3)
(547, 3)
(559, 3)
(579, 3)
(643, 3)
(679, 3)
(703, 3)
(783, 3)

[5, 11, 83, 147, 163, 207, 223, 275, 291, 303, 323, 399, 531, 547, 559, 579, 643, 679, 703, 783]


In [25]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

lst=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    ds2=digitsum(2,a)
    test=3*ds2-o2
    if (mod(a,4)==3)&(test==0):
        lst=lst+[(a-3)/4]
    if mod(a,100)==0:print(a)
print(lst)
        

100
200
300
400
500
600
700
800
900
1000
[2, 20, 36, 40, 51, 55, 68, 72, 75, 80, 99, 132, 136, 139, 144, 160, 169, 175, 195]


In [26]:
print(difference_table(lst))

[18, 16, 4, 11, 4, 13, 4, 3, 5, 19, 33, 4, 3, 5, 16, 9, 6, 20]


In [29]:
def f(n):return mod(n,6)
dt=difference_table(lst)
print(listmap(f,dt))

[0, 4, 4, 5, 4, 1, 4, 3, 5, 1, 3, 4, 3, 5, 4, 3, 0, 2]


In [42]:
for a in lst:
    print((a,ZZ(a).digits(9)))

(2, [2])
(20, [2, 2])
(36, [0, 4])
(40, [4, 4])
(51, [6, 5])
(55, [1, 6])
(68, [5, 7])
(72, [0, 8])
(75, [3, 8])
(80, [8, 8])
(99, [0, 2, 1])
(132, [6, 5, 1])
(136, [1, 6, 1])
(139, [4, 6, 1])
(144, [0, 7, 1])
(160, [7, 8, 1])
(169, [7, 0, 2])
(175, [4, 1, 2])
(195, [6, 3, 2])


In [4]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=3010)

t2=[]
t3=[]
for a in [1..3000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    ds2=digitsum(2,a)
    test2=3*ds2-o2
    if ((test2==0)&(mod(a,2)!=0)):
        t2=t2+[a]
    if mod(a,30)==0:print(a)
print();print(t2)

30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640
2670
2700
2730
2760
2790
2820
2850
2880
2910
2940
2970
3000

[5, 11, 83, 147, 163, 207, 223, 275, 291, 303, 323, 399, 531, 547, 559, 579, 643, 679, 703, 783, 1043, 1059, 1071, 1091, 1155, 1191, 1283, 1319, 1351, 1551, 1695, 1855, 1919, 2067, 2083, 2095, 2115, 2179, 2215, 2307, 2343, 2375, 2463, 2495, 2563, 2599, 2631, 2655, 2695]


In [7]:
lst=[]
for a in t2:
    b=(a+1)/2
    lst=lst+[b]
    print(a)
    print(factor(b))
    print("------------------------------------------")
print(lst)    

5
3
------------------------------------------
11
2 * 3
------------------------------------------
83
2 * 3 * 7
------------------------------------------
147
2 * 37
------------------------------------------
163
2 * 41
------------------------------------------
207
2^3 * 13
------------------------------------------
223
2^4 * 7
------------------------------------------
275
2 * 3 * 23
------------------------------------------
291
2 * 73
------------------------------------------
303
2^3 * 19
------------------------------------------
323
2 * 3^4
------------------------------------------
399
2^3 * 5^2
------------------------------------------
531
2 * 7 * 19
------------------------------------------
547
2 * 137
------------------------------------------
559
2^3 * 5 * 7
------------------------------------------
579
2 * 5 * 29
------------------------------------------
643
2 * 7 * 23
------------------------------------------
679
2^2 * 5 * 17
-----------------------------------------

In [11]:
ls=[]
for a in t2:
    ls=ls+[(a-ZZ(mod(a,3)))/3]
print(ls)
    

[1, 3, 27, 49, 54, 69, 74, 91, 97, 101, 107, 133, 177, 182, 186, 193, 214, 226, 234, 261, 347, 353, 357, 363, 385, 397, 427, 439, 450, 517, 565, 618, 639, 689, 694, 698, 705, 726, 738, 769, 781, 791, 821, 831, 854, 866, 877, 885, 898]


In [14]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..2000]: 
    if mod(a,3)==1:
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        ds3=digitsum(3,a)
        if o3!=ds3:
            t3=t3+[a]
        print((a,o3,ds3))
print(t3)

(1, 1, 1)
(4, 2, 2)
(7, 3, 3)
(10, 2, 2)
(13, 3, 3)
(16, 4, 4)
(19, 3, 3)
(22, 4, 4)
(25, 5, 5)
(28, 2, 2)
(31, 3, 3)
(34, 4, 4)
(37, 3, 3)
(40, 4, 4)
(43, 5, 5)
(46, 4, 4)
(49, 5, 5)
(52, 6, 6)
(55, 3, 3)
(58, 4, 4)
(61, 5, 5)
(64, 4, 4)
(67, 5, 5)
(70, 6, 6)
(73, 5, 5)
(76, 6, 6)
(79, 7, 7)
(82, 2, 2)
(85, 3, 3)
(88, 4, 4)
(91, 3, 3)
(94, 4, 4)
(97, 5, 5)
(100, 4, 4)
(103, 5, 5)
(106, 6, 6)
(109, 3, 3)
(112, 4, 4)
(115, 5, 5)
(118, 4, 4)
(121, 5, 5)
(124, 6, 6)
(127, 5, 5)
(130, 6, 6)
(133, 7, 7)
(136, 4, 4)
(139, 5, 5)
(142, 6, 6)
(145, 5, 5)
(148, 6, 6)
(151, 7, 7)
(154, 6, 6)
(157, 7, 7)
(160, 8, 8)
(163, 3, 3)
(166, 4, 4)
(169, 5, 5)
(172, 4, 4)
(175, 5, 5)
(178, 6, 6)
(181, 5, 5)
(184, 6, 6)
(187, 7, 7)
(190, 4, 4)
(193, 5, 5)
(196, 6, 6)
(199, 5, 5)
(202, 6, 6)
(205, 7, 7)
(208, 6, 6)
(211, 7, 7)
(214, 8, 8)
(217, 5, 5)
(220, 6, 6)
(223, 7, 7)
(226, 6, 6)
(229, 7, 7)
(232, 8, 8)
(235, 7, 7)
(238, 8, 8)
(241, 9, 9)
(244, 2, 2)
(247, 3, 3)
(250, 4, 4)
(253, 3, 3)
(256, 4, 4)
(259

(1957, 7, 7)
(1960, 8, 8)
(1963, 7, 7)
(1966, 8, 8)
(1969, 9, 9)
(1972, 6, 6)
(1975, 7, 7)
(1978, 8, 8)
(1981, 7, 7)
(1984, 8, 8)
(1987, 9, 9)
(1990, 8, 8)
(1993, 9, 9)
(1996, 10, 10)
(1999, 7, 7)
[]


In [18]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a^2*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..2000]:
    c=recipPowerConstantTerm(a)
    o3=order(3,c)
    ds3=digitsum(3,a)
    if o3!=ds3:
        t3=t3+[a]
    print((a,o3-ds3))
print(t3)

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)
(11, 0)
(12, 0)
(13, 0)
(14, 0)
(15, 0)
(16, 0)
(17, 0)
(18, 0)
(19, 0)
(20, 0)
(21, 0)
(22, 0)
(23, 0)
(24, 0)
(25, 0)
(26, 0)
(27, 0)
(28, 0)
(29, 0)
(30, 0)
(31, 0)
(32, 0)
(33, 0)
(34, 0)
(35, 0)
(36, 0)
(37, 0)
(38, 0)
(39, 0)
(40, 0)
(41, 0)
(42, 0)
(43, 0)
(44, 0)
(45, 0)
(46, 0)
(47, 0)
(48, 0)
(49, 0)
(50, 0)
(51, 0)
(52, 0)
(53, 0)
(54, 0)
(55, 0)
(56, 0)
(57, 0)
(58, 0)
(59, 0)
(60, 0)
(61, 0)
(62, 0)
(63, 0)
(64, 0)
(65, 0)
(66, 0)
(67, 0)
(68, 0)
(69, 0)
(70, 0)
(71, 0)
(72, 0)
(73, 0)
(74, 0)
(75, 0)
(76, 0)
(77, 0)
(78, 0)
(79, 0)
(80, 0)
(81, 0)
(82, 0)
(83, 0)
(84, 0)
(85, 0)
(86, 0)
(87, 0)
(88, 0)
(89, 0)
(90, 0)
(91, 0)
(92, 0)
(93, 0)
(94, 0)
(95, 0)
(96, 0)
(97, 0)
(98, 0)
(99, 0)
(100, 0)
(101, 0)
(102, 0)
(103, 0)
(104, 0)
(105, 0)
(106, 0)
(107, 0)
(108, 0)
(109, 0)
(110, 0)
(111, 0)
(112, 0)
(113, 0)
(114, 0)
(115, 0)
(116, 0)
(117, 0)
(118, 0)
(119, 0)
(120, 0)
(121, 0)
(122, 0)
(123, 0)
(

(924, 0)
(925, 0)
(926, 0)
(927, 0)
(928, 0)
(929, 0)
(930, 0)
(931, 0)
(932, 0)
(933, 0)
(934, 0)
(935, 0)
(936, 0)
(937, 0)
(938, 0)
(939, 0)
(940, 0)
(941, 0)
(942, 0)
(943, 0)
(944, 0)
(945, 0)
(946, 0)
(947, 0)
(948, 0)
(949, 0)
(950, 0)
(951, 0)
(952, 0)
(953, 0)
(954, 0)
(955, 0)
(956, 0)
(957, 0)
(958, 0)
(959, 0)
(960, 0)
(961, 0)
(962, 0)
(963, 0)
(964, 0)
(965, 0)
(966, 0)
(967, 0)
(968, 0)
(969, 0)
(970, 0)
(971, 0)
(972, 0)
(973, 0)
(974, 0)
(975, 0)
(976, 0)
(977, 0)
(978, 0)
(979, 0)
(980, 0)
(981, 0)
(982, 0)
(983, 0)
(984, 0)
(985, 0)
(986, 0)
(987, 0)
(988, 0)
(989, 0)
(990, 0)
(991, 0)
(992, 0)
(993, 0)
(994, 0)
(995, 0)
(996, 0)
(997, 0)
(998, 0)
(999, 0)
(1000, 0)
(1001, 0)
(1002, 0)
(1003, 0)
(1004, 0)
(1005, 0)
(1006, 0)
(1007, 0)
(1008, 0)
(1009, 0)
(1010, 0)
(1011, 0)
(1012, 0)
(1013, 0)
(1014, 0)
(1015, 0)
(1016, 0)
(1017, 0)
(1018, 0)
(1019, 0)
(1020, 0)
(1021, 0)
(1022, 0)
(1023, 0)
(1024, 0)
(1025, 0)
(1026, 0)
(1027, 0)
(1028, 0)
(1029, 0)
(1030, 0)
(1031,

(1751, 0)
(1752, 0)
(1753, 0)
(1754, 0)
(1755, 0)
(1756, 0)
(1757, 0)
(1758, 0)
(1759, 0)
(1760, 0)
(1761, 0)
(1762, 0)
(1763, 0)
(1764, 0)
(1765, 0)
(1766, 0)
(1767, 0)
(1768, 0)
(1769, 0)
(1770, 0)
(1771, 0)
(1772, 0)
(1773, 0)
(1774, 0)
(1775, 0)
(1776, 0)
(1777, 0)
(1778, 0)
(1779, 0)
(1780, 0)
(1781, 0)
(1782, 0)
(1783, 0)
(1784, 0)
(1785, 0)
(1786, 0)
(1787, 0)
(1788, 0)
(1789, 0)
(1790, 0)
(1791, 0)
(1792, 0)
(1793, 0)
(1794, 0)
(1795, 0)
(1796, 0)
(1797, 0)
(1798, 0)
(1799, 0)
(1800, 0)
(1801, 0)
(1802, 0)
(1803, 0)
(1804, 0)
(1805, 0)
(1806, 0)
(1807, 0)
(1808, 0)
(1809, 0)
(1810, 0)
(1811, 0)
(1812, 0)
(1813, 0)
(1814, 0)
(1815, 0)
(1816, 0)
(1817, 0)
(1818, 0)
(1819, 0)
(1820, 0)
(1821, 0)
(1822, 0)
(1823, 0)
(1824, 0)
(1825, 0)
(1826, 0)
(1827, 0)
(1828, 0)
(1829, 0)
(1830, 0)
(1831, 0)
(1832, 0)
(1833, 0)
(1834, 0)
(1835, 0)
(1836, 0)
(1837, 0)
(1838, 0)
(1839, 0)
(1840, 0)
(1841, 0)
(1842, 0)
(1843, 0)
(1844, 0)
(1845, 0)
(1846, 0)
(1847, 0)
(1848, 0)
(1849, 0)
(1850, 0)
